In [8]:
from PIL import Image,ImageOps
from cats_dogs_preproc import *
import matplotlib.pyplot as plt
%matplotlib inline
import time
import numpy as np
import os
import theano
import theano.tensor as T
import lasagne

### Волшебные переменные

In [9]:
# Изображения
IMG_GEN_IMG = False
IMG_HEIGHT = 60
IMG_WIDTH = 60
IMG_GRAY = True

# Обучаем
NUM_EPOCH = 100
BATCH_SIZE = 10

#### Препарируем изображение

In [11]:
X,y = load_data(True, IMG_HEIGHT, IMG_WIDTH, IMG_GRAY)

X_new,y_new = load_data(True, IMG_HEIGHT, IMG_WIDTH,not IMG_GRAY)

# Режем дату
TRAIN_SIZE = (len(X)*4)/5
VAL_SIZE = len(X)/10
TEST_SIZE = len(X) - TRAIN_SIZE - VAL_SIZE

X_train, y_train, X_val, y_val, X_test, y_test = prepar_data(X, y, TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

X_train_new, y_train_new, \
X_val_new, y_val_new, X_test_new, y_test_new = prepar_data(X_new, y_new, TRAIN_SIZE, VAL_SIZE, TEST_SIZE)

processed images:  1000
processed images:  2000
processed images:  3000
processed images:  4000
processed images:  5000
processed images:  6000
processed images:  7000
processed images:  8000
processed images:  9000
processed images:  10000
processed images:  11000
processed images:  12000
processed images:  13000
processed images:  14000
processed images:  15000
processed images:  16000
processed images:  17000
processed images:  18000
processed images:  19000
processed images:  20000
processed images:  21000
processed images:  22000
processed images:  23000
processed images:  24000
processed images:  25000
processed images:  1000
processed images:  2000
processed images:  3000
processed images:  4000
processed images:  5000
processed images:  6000
processed images:  7000
processed images:  8000
processed images:  9000
processed images:  10000
processed images:  11000
processed images:  12000
processed images:  13000
processed images:  14000
processed images:  15000
processed images: 

In [24]:
input_X = T.tensor4("X cat/dog image")
target_y = T.vector("target Y cat/dog",dtype='int32')
input_shape = [None] + list(X.shape[1:])

input_X_new = T.tensor4("X cat/dog image")
input_shape_new = [None] + list(X_new.shape[1:])

print X.shape, X_new.shape

(25000, 1, 60, 60) (25000, 3, 60, 60)


#### Разварачиваем сеть

In [17]:
#
#
#    FIRST INPUT
#
#

input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

conv_1 = lasagne.layers.Conv2DLayer(input_layer, num_filters = 10,filter_size = (2,2),
                                    nonlinearity= lasagne.nonlinearities.very_leaky_rectify)

batch_1 = lasagne.layers.batch_norm(conv_1)



#
#
#    SECOND INPUT
#

input_layer2 = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X_new)

conv_2 = lasagne.layers.Conv2DLayer(input_layer2, num_filters = 10,filter_size = (2,2),
                                    nonlinearity= lasagne.nonlinearities.very_leaky_rectify)

batch_21 = lasagne.layers.batch_norm(conv_2)

#
#
#     OUTPUT
#


con = lasagne.layers.ConcatLayer([batch_1, batch_21])



pool_2 = lasagne.layers.MaxPool2DLayer(con, pool_size = 5)

batch_22 = lasagne.layers.batch_norm(pool_2)


dense_output = lasagne.layers.DenseLayer(batch_22, num_units = 2, nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')

In [27]:
#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(dense_output)

#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)

#функция ошибки - средняя кроссэнтропия
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()

#регуляризация, если захотите
loss += lasagne.regularization.l2(loss)

#<возможно добавить регуляризатор>
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#сразу посчитать словарь обновлённых значений с шагом по градиенту, как раньше
updates_sgd = lasagne.updates.adamax(loss,all_weights,learning_rate = 0.0001)

#функция, которая обучает сеть на 1 шаг и возвращащет значение функции потерь и точности
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#функция, которая считает точность
accuracy_fun = theano.function([input_X,target_y],accuracy)

MissingInputError: An input of the graph, used to compute AbstractConv2d{border_mode='valid', subsample=(1, 1), filter_flip=True, imshp=(None, 1, 60, 60), kshp=(10, 1, 2, 2)}(X cat/dog image, W), was not provided and not given a value.Use the Theano flag exception_verbosity='high',for more information on this error.

Backtrace when the variable is created:
  File "/root/miniconda/envs/rep_py2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 260, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/root/miniconda/envs/rep_py2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 212, in dispatch_shell
    handler(stream, idents, msg)
  File "/root/miniconda/envs/rep_py2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 370, in execute_request
    user_expressions, allow_stdin)
  File "/root/miniconda/envs/rep_py2/lib/python2.7/site-packages/ipykernel/ipkernel.py", line 175, in do_execute
    shell.run_cell(code, store_history=store_history, silent=s

#### Обучаем

In [ ]:
train = []
test = []
for epoch in range(NUM_EPOCH):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, BATCH_SIZE):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, BATCH_SIZE):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, NUM_EPOCH, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    train.append(train_acc / train_batches * 100)
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    test.append(val_acc / val_batches * 100)

In [ ]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))
# if(max_accuracy<(test_acc / test_batches * 100)):
#     max_accuracy = test_acc / test_batches * 100

# max_accuracy = test_acc / test_batches * 100
# print max_accuracy

# Замер скорости сети

In [ ]:
# weight = np.load('the weight of the networks/xenx_network_v_1_baseline_weight.npy')
# lasagne.layers.set_all_param_values(dense_output, weight)
# weight = lasagne.layers.get_all_param_values(dense_output)
# np.save('the weight of the networks/xenx_network_v_1_baseline_weight.npy', weight)

In [ ]:
net = theano.compile.function([input_layer.input_var], y_predicted)

In [ ]:
all_run_time = 0 
result = 0 
count = 1000
full_start_time = time.time() 
for i in range(count):
    start_time = time.time() 
    res = net(X_test[:1]) 
    run_time = time.time() - start_time 
    all_run_time += run_time 
    # print run_time 
    # print res 
    result += int(1/run_time) 
    # print int(1/run_time)
run_time = time.time() - full_start_time
print 'Result:' 
print '\tRun time:\t\t{:.3f}'.format(run_time)
print '\tAverage time:\t\t{:.5f}'.format(all_run_time/count) 
print '\tPer second:\t\t{:.5f}'.format(count/run_time)